In [93]:
# Essentials
import numpy as np
import pandas as pd
import datetime
import random

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Misc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000

In [94]:
bp_file = '/mnt/volume1/T2D/EMR-data/OralGlucose.csv'
bp_file_p = '/mnt/volume1/T2D/EMR-data2/OralGlucose.csv'

In [95]:
data_pd = pd.read_csv(bp_file, sep='|')


In [96]:
len(data_pd), len(pd.Series(data_pd["Deidentified_Pat_ID"].tolist()).sort_values().unique())

(1649, 123)

In [97]:
data_pd.head()

,Deidentified_Pat_ID,ResultNum,Component,Result,ResultedYear
0,ZHAZTNOZ,1,GLUCOSE FASTING,96,2014
1,ZHAZTNOZ,1,FASTING,Yes,2014
2,ZHAZTNOZ,1,DOSE GIVEN,100 gm Glucose,2014
3,ZHAZTNOZ,1,TIME GIVEN,7:45 AM,2014
4,ZHAZTNOZ,5,GLUCOSE 1 HR,178,2014


In [98]:
pd.Series(data_pd["Component"].tolist()).sort_values().unique()

array(['DOSE GIVEN', 'FASTING', 'GLU TOL BEVERAGE DOSE',
       'GLU TOL BEVERAGE EXP DATE', 'GLU TOL BEVERAGE LOT #',
       'GLUCOSE 1 HR', 'GLUCOSE 2 HR', 'GLUCOSE 3 HR', 'GLUCOSE FASTING',
       'HEMOLYSIS INDEX', 'ICTERIC INDEX', 'LIPEMIC INDEX', 'TIME GIVEN'],
      dtype=object)

In [99]:
HEMOLYSIS_dt = data_pd[(data_pd['Component'] == 'HEMOLYSIS INDEX')]
HEMOLYSIS_dt = HEMOLYSIS_dt[(HEMOLYSIS_dt['Component'].notna())]
len(HEMOLYSIS_dt)

278

In [100]:
len(data_pd[(data_pd['Component'] == 'HEMOLYSIS INDEX')])

278

In [101]:
data_pd = data_pd[~(data_pd['Component'].isna())]
data_pd = data_pd[~(data_pd['Result'].isna())]
data_pd = data_pd[~(data_pd['Component'] == 'FASTING')]
data_pd = data_pd[~(data_pd['Component'] == 'DOSE GIVEN')]
data_pd = data_pd[~(data_pd['Component'] == 'TIME GIVEN')]
data_pd = data_pd[~(data_pd['Component'] == 'GLU TOL BEVERAGE EXP DATE')]
data_pd = data_pd[~(data_pd['Component'] == 'GLU TOL BEVERAGE DOSE')]
data_pd = data_pd[~(data_pd['Component'] == 'GLU TOL BEVERAGE LOT #')]

#data_pd = data_pd[~(data_pd['Component'] == 'HEMOLYSIS INDEX')]
#data_pd = data_pd[~(data_pd['Component'] == 'ICTERIC INDEX')]
#data_pd = data_pd[~(data_pd['Component'] == 'LIPEMIC INDEX')]

data_pd.reset_index(inplace=True)

data_pd.sort_values(["Deidentified_Pat_ID", "ResultedYear", 'Component'], inplace=True)

In [102]:
len(data_pd)

739

In [108]:
sinlge_data1 = data_pd.drop_duplicates(["Deidentified_Pat_ID", "ResultedYear", 'Component'])
len(data_pd)

739

In [106]:
data_pd[['Result']] = data_pd[['Result']].astype('float')
data_bp_mean1 = data_pd.groupby(['Deidentified_Pat_ID', 'ResultedYear', 'Component'])[['Result']].max()
data_bp_mean1.reset_index(inplace=True)
data_bp_mean1.head

<bound method NDFrame.head of     Deidentified_Pat_ID  ResultedYear        Component  Result
0              MNVHNHNZ          2015     GLUCOSE 1 HR   175.0
1              MNVHNHNZ          2015     GLUCOSE 2 HR   142.0
2              MNVHNHNZ          2015     GLUCOSE 3 HR   125.0
3              MNVHNHNZ          2015  GLUCOSE FASTING    87.0
4              MNVHNHNZ          2016     GLUCOSE 1 HR   144.0
5              MNVHNHNZ          2016     GLUCOSE 2 HR   100.0
6              MNVHNHNZ          2016     GLUCOSE 3 HR    97.0
7              MNVHNHNZ          2016  GLUCOSE FASTING    88.0
8                ZADTOK          2016     GLUCOSE 2 HR    74.0
9                ZADTOK          2016  GLUCOSE FASTING    90.0
10               ZADTOK          2016  HEMOLYSIS INDEX    15.0
11               ZADTOK          2016    ICTERIC INDEX     2.0
12               ZADTOK          2016    LIPEMIC INDEX    20.0
13              ZAVKNHD          2017     GLUCOSE 1 HR    84.0
14              ZAVKNHD  

In [109]:
len(pd.Series(data_bp_mean1["Deidentified_Pat_ID"].tolist()).sort_values().unique())

123

In [110]:
new_data = data_bp_mean1.set_index(['Deidentified_Pat_ID', 'ResultedYear', 'Component']).unstack(level=2)

In [113]:
new_data.reset_index(inplace=True)
len(new_data)

140

In [114]:
new_data.head()

index Deidentified_Pat_ID ResultedYear       Result               \
Component                                        GLUCOSE 1 HR GLUCOSE 2 HR   
0             0            MNVHNHNZ         2015        175.0        142.0   
1             1            MNVHNHNZ         2016        144.0        100.0   
2             2              ZADTOK         2016          NaN         74.0   
3             3             ZAVKNHD         2017         84.0        119.0   
4             4             ZAVOOHT         2014        166.0        157.0   

                                                                      \
Component GLUCOSE 3 HR GLUCOSE FASTING HEMOLYSIS INDEX ICTERIC INDEX   
0                125.0            87.0             NaN           NaN   
1                 97.0            88.0             NaN           NaN   
2                  NaN            90.0            15.0           2.0   
3                 81.0            79.0             NaN           NaN   
4                 86.0            82.0             NaN           NaN   

                         
Component LIPEMIC INDEX  
0                   NaN  
1                   NaN  
2                  20.0  
3                   NaN  
4                   NaN

In [119]:
new_data.sort_values(["Deidentified_Pat_ID", 'ResultedYear'], inplace=True)
new_data.columns = ['in',"Deidentified_Pat_ID", 'ResultedYear','GLUCOSE 1 HR','GLUCOSE 2 HR','GLUCOSE 3 HR','GLUCOSE FASTING','HEMOLYSIS INDEX','ICTERIC INDEX','LIPEMIC INDEX']
new_data = new_data.drop_duplicates(['Deidentified_Pat_ID'], keep='last')
len(new_data)

123

In [121]:
new_data = new_data[["Deidentified_Pat_ID",'GLUCOSE 1 HR','GLUCOSE 2 HR','GLUCOSE 3 HR','GLUCOSE FASTING','HEMOLYSIS INDEX','ICTERIC INDEX','LIPEMIC INDEX']]

In [124]:
new_data.head()

,Deidentified_Pat_ID,GLUCOSE 1 HR,GLUCOSE 2 HR,GLUCOSE 3 HR,GLUCOSE FASTING,HEMOLYSIS INDEX,ICTERIC INDEX,LIPEMIC INDEX
1,MNVHNHNZ,144.0,100.0,97.0,88.0,NaN,NaN,NaN
2,ZADTOK,NaN,74.0,NaN,90.0,15.0,2.0,20.0
3,ZAVKNHD,84.0,119.0,81.0,79.0,NaN,NaN,NaN
6,ZAVOOHT,160.0,165.0,131.0,81.0,NaN,NaN,NaN
7,ZDAKGNH,NaN,172.0,NaN,87.0,NaN,NaN,NaN


In [123]:
new_data.to_csv(bp_file_p, index=False)